In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd 'drive/MyDrive/UT Austin/Courses/SLT/PS3'

/content/drive/MyDrive/UT Austin/Courses/SLT/PS3


In [50]:
from torch._C import AliasDb
import librosa
import math
import numpy as np
import scipy.signal
from scipy.special import logsumexp
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, (1, 5))
        self.fc1 = nn.Linear(128, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 48)
        self.sm = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 128)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sm(x)
        return x

def load_audio_to_melspec_tensor(wavpath, sample_rate=16000):
    window_size = .025
    window_stride = 0.01
    n_dft = 512
    win_length = int(sample_rate * window_size)
    hop_length = int(sample_rate * window_stride)
    y, sr = librosa.load(wavpath, sr=sample_rate)
    y = y - y.mean()
    y = np.append(y[0],y[1:]-.97*y[:-1])
    # compute mel spectrogram
    stft = librosa.stft(y, n_fft=n_dft, hop_length=hop_length,
        win_length=win_length, window=scipy.signal.hamming)
    spec = np.abs(stft)**2
    mel_basis = librosa.filters.mel(sr=sample_rate, n_fft=n_dft, n_mels=40, fmin=20)
    melspec = np.dot(mel_basis, spec)
    logspec = librosa.power_to_db(melspec, ref=np.max)
    logspec = np.transpose(logspec)
    logspec_tensor = torch.tensor(logspec)
    return logspec_tensor

def compute_phone_likelihoods(model, logspec):
    likelihood_list = []
    with torch.no_grad():
        for j in range(6, logspec.size(0) - 5):
            inp = logspec[j-5:j+6,:].unsqueeze(0)
            output = model(inp) # output will be log probabilities over classes
            output = output - math.log(1. / 48) # subtract the logprob of the class priors (assumed to be uniform)
            likelihood_list.append(output[0])
    likelihoods = torch.transpose(torch.stack(likelihood_list, dim=1), 0, 1).numpy()
    return likelihoods

class MyHMM:
    def __init__(self, state_labels, initial_state_distribution, transition_matrix, eps=1e-200):
        self.eps = eps
        self.pi = np.log(initial_state_distribution + eps)
        self.A = np.log(transition_matrix + eps) #A_{ji} is prob of transitioning from state j to state i
        self.labels = state_labels # a list where self.labels[j] is the index of the phone label belonging to the jth state
        self.N_states = len(self.labels)

    def forward(self, state_likelihoods):
        # state_likelihoods.shape is assumed to be (N_timesteps, 48)
        # TODO: fill in
        # self.N_states = i = 4
        # t = N_timesteps = 100
        # second dimention of state_likelihoods is the log likelihoods accross
        # the 48 different phoneme classes

        # defining alpha, shape (100, 4), alpha[0].shape = 4
        alpha = np.zeros((len(state_likelihoods), self.N_states))
        # initialize alpha[0][i]=alpha_0(i) for i = 4 states
        for i in range(self.N_states):
          alpha[0][i] = state_likelihoods[0][self.labels[i]] + self.pi[i]

        for t in range(1, len(state_likelihoods)): # 1,...,99
          for i in range(self.N_states):
            # TODO: confirm if it is over 48 or 4, it should be self.N_states
            summ = np.zeros(self.N_states)
            for j in range(self.N_states):
              summ[j] += alpha[t-1][j] + self.A[j][i]
            summation = logsumexp(summ)
            alpha[t][i] = summation + state_likelihoods[t][self.labels[i]]
        return alpha[len(state_likelihoods) - 1][self.N_states - 1]

    def viterbi(self, state_likelihoods):
        # state_likelihoods.shape is assumed to be (N_timesteps, 48)
        # TODO: fill in
        delta = np.zeros((len(state_likelihoods), self.N_states)) # shape: (100, 4) = (T, N)
        # defining matrix
        psi = np.zeros((len(state_likelihoods), self.N_states)) # shape: (100, 4) = (T, N)
        # initializing
        for i in range(self.N_states):
          delta[0][i] = state_likelihoods[0][self.labels[i]] + self.pi[i]
        # induction step
        for t in range(1, len(state_likelihoods)):
          for i in range(self.N_states):
            summ = np.zeros(self.N_states)
            max = 0
            for j in range(self.N_states):
              summ[j] += delta[t - 1][j] + self.A[j][i]
            max = np.amax(summ)
            delta[t][i] = max + state_likelihoods[t][self.labels[i]]
            psi[t][i] = np.argmax(summ)
        # score best path
        p = np.amax(delta[len(state_likelihoods) - 1])
        # best final state
        q = np.zeros(len(state_likelihoods)).astype(np.int32)
        q[len(state_likelihoods) - 1] = np.argmax(delta[len(state_likelihoods) - 1])
        # backtracking
        for t in range(len(state_likelihoods) - 2, -1, -1):
          q[t] = psi[t + 1][q[t + 1]]
        return q

    def viterbi_transition_update(self, state_likelihoods):
        # state_likelihoods.shape is assumed to be (N_timesteps, 48)
        # TODO: fill
        # hidden states should be 4 or 6

        p = self.viterbi(state_likelihoods)
        eps = 1e-200
        self.pi = np.zeros(self.N_states)
        # self.pi = self.pi + eps

        for i in range(self.N_states):
          if p[0] == i:
            self.pi[i] += 1
          else:
            self.pi[i] += eps
          self.pi[i] = np.log(self.pi[i])

        for i in range(self.N_states):
          for j in range(self.N_states):
            numerator = 0
            denominator = 0
            for t in range(len(state_likelihoods) - 1):
              if t < len(state_likelihoods) - 1 and p[t] == i and p[t + 1] == j:
                numerator += 1
              if p[t] == i:
                denominator += 1
            self.A[i][j] = np.log(numerator/denominator + eps)

        return

model = MyNet()
model.load_state_dict(torch.load('lab3_AM.pt'))

lab3_data = np.load('lab3_phone_labels.npz')

phone_labels = list(lab3_data['phone_labels'])

def phones2indices(phones):
    return [phone_labels.index(p) for p in phones]

fee_HMM = MyHMM(phones2indices(['sil', 'f', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))
pea_HMM = MyHMM(phones2indices(['sil', 'p', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))
rock_HMM = MyHMM(phones2indices(['sil', 'r', 'aa', 'cl', 'k', 'sil']), np.array([0.5,0.5,0,0,0,0]), np.array([[.9,.1,0,0,0,0],[0,.9,.1,0,0,0],[0,0,.9,.1,0,0],[0,0,0,.9,.1,0],[0,0,0,0,.9,.1],[0,0,0,0,0,1]]))
burt_HMM = MyHMM(phones2indices(['sil', 'b', 'er', 'cl', 't', 'sil']), np.array([0.5,0.5,0,0,0,0]), np.array([[.9,.1,0,0,0,0],[0,.9,.1,0,0,0],[0,0,.9,.1,0,0],[0,0,0,.9,.1,0],[0,0,0,0,.9,.1],[0,0,0,0,0,1]]))
see_HMM = MyHMM(phones2indices(['sil', 's', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))
she_HMM = MyHMM(phones2indices(['sil', 'sh', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))

# TODO: write your code to use your HMMs below here (or in a new block)

In [51]:
# computing the likelihoods
burt_likelihoods = compute_phone_likelihoods(model, load_audio_to_melspec_tensor("burt.wav"))
fee_likelihoods = compute_phone_likelihoods(model, load_audio_to_melspec_tensor("fee.wav"))
pea_likelihoods = compute_phone_likelihoods(model, load_audio_to_melspec_tensor("pea.wav"))
rock_likelihoods = compute_phone_likelihoods(model, load_audio_to_melspec_tensor("rock.wav"))
see_likelihoods = compute_phone_likelihoods(model, load_audio_to_melspec_tensor("see.wav"))
she_likelihoods = compute_phone_likelihoods(model, load_audio_to_melspec_tensor("she.wav"))

hmms = [fee_HMM, pea_HMM, rock_HMM, burt_HMM, see_HMM, she_HMM]
wavs = [fee_likelihoods, pea_likelihoods, rock_likelihoods, burt_likelihoods, see_likelihoods, she_likelihoods]

likelihood_table = np.zeros((6,6))

for i in range(len(wavs)):
  for j in range(len(hmms)):
    likelihood_table[i, j] = (hmms[j].forward(wavs[i]))
likelihood_table = np.around(likelihood_table, 2)
print(likelihood_table)

[[ 208.75  179.36  -92.06  -85.49  192.24  182.34]
 [ 250.02  268.16    7.83   71.11  236.17  234.84]
 [ -62.59   -5.84  158.39   66.67  -61.83  -63.55]
 [ -62.84  -19.97  116.55  221.06  -75.79 -100.31]
 [  72.8    75.08 -177.84 -160.09  229.06  123.82]
 [  78.29   91.94 -211.49 -191.97  124.63  281.11]]


In [52]:
optimal_hidden_state_rock = rock_HMM.viterbi(rock_likelihoods)
print(optimal_hidden_state_rock)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2
 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5]


In [53]:
first_transition_matrix = np.around(np.exp(rock_HMM.A), 2)
print(first_transition_matrix)
first_likelihoods_rock = rock_HMM.forward(rock_likelihoods)
print("likelihood 1: ", first_likelihoods_rock)

[[0.9 0.1 0.  0.  0.  0. ]
 [0.  0.9 0.1 0.  0.  0. ]
 [0.  0.  0.9 0.1 0.  0. ]
 [0.  0.  0.  0.9 0.1 0. ]
 [0.  0.  0.  0.  0.9 0.1]
 [0.  0.  0.  0.  0.  1. ]]
likelihood:  158.3910544157233


In [54]:
# Viterbi update
rock_HMM.viterbi_transition_update(rock_likelihoods)
second_transition_matrix = np.around(np.exp(rock_HMM.A), 2)
print(second_transition_matrix)
# computing likelihood
second_likelihoods_rock = rock_HMM.forward(rock_likelihoods)
print("likelihood 2: ", second_likelihoods_rock)

[[0.93 0.07 0.   0.   0.   0.  ]
 [0.   0.92 0.08 0.   0.   0.  ]
 [0.   0.   0.92 0.08 0.   0.  ]
 [0.   0.   0.   0.92 0.08 0.  ]
 [0.   0.   0.   0.   0.8  0.2 ]
 [0.   0.   0.   0.   0.   1.  ]]
likelihood 2:  159.5448084880059


In [58]:
print("First transition matrix: \n", first_transition_matrix)
print("Second transition matrix: \n", second_transition_matrix)
print("Likelihood 1: ", first_likelihoods_rock)
print("Likelihood 2: ", second_likelihoods_rock)

First transition matrix: 
 [[0.9 0.1 0.  0.  0.  0. ]
 [0.  0.9 0.1 0.  0.  0. ]
 [0.  0.  0.9 0.1 0.  0. ]
 [0.  0.  0.  0.9 0.1 0. ]
 [0.  0.  0.  0.  0.9 0.1]
 [0.  0.  0.  0.  0.  1. ]]
Second transition matrix: 
 [[0.93 0.07 0.   0.   0.   0.  ]
 [0.   0.92 0.08 0.   0.   0.  ]
 [0.   0.   0.92 0.08 0.   0.  ]
 [0.   0.   0.   0.92 0.08 0.  ]
 [0.   0.   0.   0.   0.8  0.2 ]
 [0.   0.   0.   0.   0.   1.  ]]
Likelihood 1:  158.3910544157233
Likelihood 2:  159.5448084880059
